# Homework 5. Transformers for token classification
Max Base Points: 15
- [x] Introduce fixes, mentioned in `TODO`s
- [x] Your model also have to handle ukrainian and russian languages. Make sure to add `../data/mantis_analytics_location_detection/ru_geo_dataset.csv` (from https://www.kaggle.com/datasets/vladimirsydor/mantis-analytics-location-detection/data). Think about
    - New Validation
    - New Word Embeddings
    - Maybe separate models
    - IMPORTANT: Take into account `doc_id`
    - Take into account that markup is far from ideal. Maybe pre-processing may help
- [ ] Coming back to Lecture 3. F1 is the final production metric but it strongly depends on threshold. Maybe you can use some "soft" metric for model comparison ? 
- [x] Do we need additional post-processing ?
- [x] Tune `BERT` / `DEBERTA` or other bert-like models on the whole dataset for more epochs, maybe re-write it. Tune all other hyper-params.
- [ ] Try different optimizers lr schedulers 
- [ ] Implement Cross-Validation and add test houldout
- [x] Make a submit to Kaggle
- [ ] Additional points: first private score - 25 points, second private score - 15 points, third private score - 10 points
- [ ] Do not hesitate to use `Discussion` and `Code` on Kaggle. All additional useful insights will be also granted with additional scores
- [x] If you do not want to share with other competitors - you can share with lectors. It can be useful for Mantis usecase 

## General thoughts and approaches 

I thought of two possible approaches to deal with the two-languages nature of the test data:
- Train two separate models for the uk and ru NER, on the inference detect the language of the sample with auxiliary model first and apply correspondent trained model after
- Train multilingual model on the dataset which includes samples from both languages, with the same proportion as in the test data 

At first glance it's not obvious which one should work better. Inserting step with language detection adds one more possible source of error, which may be a bottleneck, because if language detected incorrectly, it will be supplied to the model for another one and likely won't output meaningful result. On the other hand, specialized models expected to have much better performance on the one language than the general one, so high performance on the correctly detected samples may compensate poor performance on the wrong one. 

Taking this into account, I decided to try both approaches.

## Combined multibert

- combined multibert, balanced: [train](multibert_combined_train_1.ipynb), [inference]()

Datasets sampling: [create_light_versions.ipynb](create_light_versions.ipynb)

## Separated models 

- Lang detection experiments: [lang_detection](lang_detection.ipynb)

- Uk-roberta: [train](roberta_uk.ipynb), [inference]()

- Uk-multibert + ru-multibert: [train uk](), [train ru](multibert_ru.ipynb), [inference]()

## Validation nuances

## Some additional insights

1. During the inspection of prediction of one of the models, I noticed that huggingface's token classification pipeline unites some punctuation tokens with spaces around, when they shouldn't be there. It concerns three variants of ukrainian apostroph and hyphen  (which sometimes appear in the address string). For example, model correctly recognized entity `вул. Л. Лукʼяненка, 13`, but in the final prediction it was represented as `вул. Л. Лук ʼ яненка, 13`. To deal with it, I added the next regex to the post-processing stage. It finds mentioned characters in the string and removes spaces around it. Also, this snippet removes entities which contain "#", as it indicates that the model took only part of the word.
```python
if ent['score'] >= th and "#" not in ent['word']:
    word = re.sub(r"\s*([ʼ\''-])\s*", r"\1", ent['word'])
    res.append(word)
```
This adds `0.02` points to LB score. I suspect that the same problem appears in the `nlp_ua_bert_baseline.csv`, as I had exactly the same score with it at some point.

**ToDo** think about more generalized approach -- match classified tokens with the original text and for the prediction return a substring of the original text

**ToDo** instead of removing entities with "#" try to attach previous missing token to them

2. Test set consists of messages from telegram channels and contains lots of emojis. I noticed that sometimes tokens with emojis are being recognized as entities, yet as I get from the `labeling_sample.csv` inspection it shouldn't be the case. For example, this concertns channels' names, which sometimes have emoji and city/region name. Removing entities which contain emoji boosted LB score a bit too. Although it doesn't solve fully problem with telegram channels names, and probably it's possible to train model to distinguish them from the location entities, by adding correspondent data to the dataset and introducing additional NER labels 'B-TG', 'I-TG'.